# 키토크 유사동의어

In [1]:
# 데이터 처리
import re   # 정규표현식 
import numpy as np   # 넘파이
import pandas as pd   # 판다스

# 경고 안뜨게
import warnings
warnings.filterwarnings(action='ignore')

### DB에서 데이터 받아오기

In [2]:
import pymysql
from sqlalchemy import create_engine

host_url = ### 입력 ### 
user_nm = ### 입력 ### 
passwd = ### 입력 ### 
port_num =  ### 입력 ### 
db_name = ### 입력 ### 
engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8')
engine_conn = engine.connect()


keytalk_list = pd.read_sql("""SELECT kl_pk, kl_name FROM keytalk_list;""", engine_conn)
engine_conn.close()

원본 유지

In [3]:
keytalk_list['keytalk'] = keytalk_list['kl_name']

### preprocessing

1. 소문자로

In [4]:
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.lower()

2. 특수기호 & 외국어

In [5]:
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('-', ' ')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('á', 'a')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('å', 'a')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('í', 'i')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ü', 'u')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ú', 'u')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ë', 'e')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('é', 'e')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ö', 'o')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ø', 'o')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ô', 'o')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ō', 'o')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('ó', 'o')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('+', ' plus')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('&', 'and')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('æ', 'ae')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('%', ' percent')

com = re.compile('[,\'.]')
keytalk_list['keytalk'] = keytalk_list['keytalk'].apply(lambda x: com.sub('', x))

3. 숫자를 발음나는대로

In [6]:
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('1980', 'nineteen eighty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('1990', 'nineteen ninety')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2000', 'two thousand')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2010', 'two thousand ten')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2015', 'two thousand fifteen')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2016', 'two thousand sixteen')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2017', 'two thousand seventeen')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('2018', 'two thousand eighteen')

keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('13', 'thirteen')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('17', 'seventeen')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('21', 'twenty one')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('30', 'thirty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('60', 'sixty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('70', 'seventy')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('80', 'eighty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('90m', 'to ninety minutes')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('90', 'ninety')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('100', 'hundred')

keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('120m', 'to one hundred twenty minutes')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('120', 'one hundred twenty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('150m', 'to one hundred fifty minutes')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('150', 'one hundred fifty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('50', 'fifty')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('5', 'five')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('6', 'six')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('7', 'seven')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('8', 'eight')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('9', 'nine')
keytalk_list['keytalk'] = keytalk_list['keytalk'].str.replace('10', 'ten')


## 해당 단어의 Frequency

Frequency Score : 단어의 사용빈도에 따라 점수로 나타낸 값. 높을 수록 사용 빈도가 높다. 

In [7]:
import json 
import requests
from bs4 import BeautifulSoup

vowels = ['a', 'e', 'i', 'o', 'u']

def word_frequency(i):
    original = keytalk_list['kl_name'][i]
    
    # 단어 끝이 모음이면 s를 붙인 후 검색
    if original[:-1] in (vowels):
        keytalk_name = original + 's'
        
    # 단어 끝이 자음이면 e를 붙인 후 검색        
    else:
        keytalk_name = original + 'e'
 
    # get request
    rq = requests.get(f'https://api.datamuse.com/words?md=f&sl={keytalk_name}&max=10')
            
    # html 가져오기
    html = rq.text

    # json_데이터프레임으로 변경
    rhyme_json_df = pd.DataFrame(json.loads(html))
    rhyme_json_df['tags'] = rhyme_json_df['tags'].apply(lambda x: float(x[0].replace('f:', '')))

    # original에 해당하는 것
    rhyme_json_df = rhyme_json_df[(rhyme_json_df['word']==original)]

    # 존재하지 않으면 빈칸
    if len(rhyme_json_df) == 0:
        return_value = ''
                
    # 존재하면 frequency 값 가져오기
    else:
        return_value = rhyme_json_df['tags']
        
    return return_value

In [8]:
word_frequency(1)

0    110.579752
Name: tags, dtype: float64

In [ ]:
from multiprocessing import Pool 

if __name__=='__main__':
    pool = Pool(processes=10)
    result = pool.map(rhyme_Zone, range(0, len(keytalk_list)), 1000)
    
keytalk_list['kl_frequency'] = result

### Rhyme Zone에서 유사동의어 가져오기

In [9]:
import json 
import requests
import xmltodict
from bs4 import BeautifulSoup


def rhyme_Zone(i):  
    original = keytalk_list['kl_name'][i]
    
    while True:
        error = 0
        try:
            # get request
            rq = requests.get(f'https://api.datamuse.com/words?md=f&sl={original}&max=10')

            # html 가져오기
            html = rq.text

            # json으로 변경
            rhyme_json_df = pd.DataFrame(json.loads(html))
            
            # 조건
            rhyme_json_df['tags'] = rhyme_json_df['tags'].apply(lambda x: float(x[0].replace('f:', '')))
            
            # 1. Direct Replacement
            filter1 = rhyme_json_df[(rhyme_json_df['tags']<=20)&(
                                                rhyme_json_df['score']==100)&(
                                                        rhyme_json_df['word']!=original)]           
            # 2. Dictionary
            filter2 = rhyme_json_df[(rhyme_json_df['tags']>=0)&(
                                        rhyme_json_df['score']>=85)&(
                                            rhyme_json_df['word']!=original)]
            
            # 1에 이미 들어있는거라면 2에서 제외하기
            try:
                in_f1 = list(set(filter2['word'])&set(filter1['word']))[0]
                filter2 = filter2[filter2['word']!=in_f1]
            except:
                pass
            
            # numSyllables 제거
            if len(filter1)>0:
                filter1 = filter1.drop(['numSyllables'], axis = 1)
                filter1 = filter1.to_dict(orient='records')
            else:
                filter1 = ''
    
            if len(filter2)>0:
                filter2 = filter2.drop(['numSyllables'], axis = 1)
                filter2 = filter2.to_dict(orient='records')
            else:
                filter2 = ''
                
        except Exception as e:
            print('Error :', i, e)
            return_value = 'Error'
        else:
            break
    return filter1, filter2

In [10]:
rhyme_Zone(1)

([{'word': 'intrusting', 'score': 100, 'tags': 0.021413}],
 [{'word': 'untrusting', 'score': 95, 'tags': 0.051473},
  {'word': 'entrusting', 'score': 95, 'tags': 0.532295},
  {'word': 'interrupting', 'score': 85, 'tags': 3.169933},
  {'word': 'intreating', 'score': 85, 'tags': 0.012719}])

In [ ]:
from multiprocessing import Pool
import sys

if __name__=='__main__':
    pool = Pool(processes=10)
    result = pool.map(rhyme_Zone, range(0, len(keytalk_list)), 1000)
        
result = pd.DataFrame(result, columns = ['filter1', 'filter2'])
keytalk_list = pd.concat([keytalk_list, result], axis = 1)

## WordNet Check

WordNet 사이트에 존재하는지 확인

In [11]:
def wordnet(word):
    url = f'http://wordnetweb.princeton.edu/perl/webwn?s={word}&sub=Search+WordNet&o2=1&o0=1&o8=1&o1=1&o7=1&o5=1&o9=&o6=1&o3=1&o4=1&h='
    
    rq = requests.get(url)
    
    # html 가져오기
    html = rq.text
      
    # html 파싱하기  
    bs = BeautifulSoup(html, 'html.parser')
    
    # 결과가 존재하는지 확인
    if bs.find('h3').text == 'Your search did not return any results.':
        check = 'None'
    else:
        check = 'Check'
        
    return check

def wordnet_check(i, n):
    if keytalk_list[f'filter{n}'][i]!='':
        keytalk_list[f'filter{n}'][i] = [w for w in keytalk_list[f'filter{n}'][i] if wordnet(w.get('word'))=='Check']
    else:
        keytalk_list[f'filter{n}'][i] = ''

In [ ]:
from tqdm import tqdm_notebook

for k in tqdm_notebook(range(0, len(keytalk_list))):
    wordnet_check(k, 1)

In [ ]:
from tqdm import tqdm_notebook

for k in tqdm_notebook(range(0, len(keytalk_list))):
    wordnet_check(k, 2)

## Score 만들기

score가 100이상이면 100으로 바꾸기

In [ ]:
# score가 100이상이면 100으로 바꾸기
def score_to(keytalk):
    [rz.update({'score':100}) for rz in keytalk if rz.get('score') > 100]      
    return keytalk

keytalk_list['filter1'] = list(map(lambda x: score_to(x), keytalk_list['filter1']))
keytalk_list['filter2'] = list(map(lambda x: score_to(x), keytalk_list['filter2']))

filter별로 구분하여 Score 구하기

In [ ]:
# filter별로 구분
filter1_df = keytalk_list[['kl_pk', 'kl_name', 'filter1']]
filter2_df = keytalk_list[['kl_pk', 'kl_name', 'filter2']]


# 시그모이드
def sig(x):
    return 1/(1+np.exp(-x))

# Score 구하기
filter1_df['similar_score'] = list(map(lambda x: sum([s.get('tags') for s in x]), filter1_df['filter1']))
filter2_df['similar_score'] = list(map(lambda x: sum([s.get('score')*(sig(s.get('tags'))**2) for s in x]), filter2_df['filter2']))

In [ ]:
def get_score(i):
    score_1 = sum([(f.get('score')*(sig(f.get('tags'))**2)) for f in keytalk_list['filter1'][i]])
    score_2 = sum([(f.get('score')*(sig(f.get('tags'))**2)) for f in keytalk_list['filter2'][i]])
    return score_1 + score_2

keytalk_list['similar_score'] = list(map(get_score, range(0, len(keytalk_list))))
keytalk_list = keytalk_list.sort_values(['similar_score'], ascending = False).reset_index(drop = True)

### 정리

딕셔너리로 있는 것을 행으로 바꾸기  

In [ ]:
def dict_to_row(i, n):
    if n == 1:
        df = filter1_df
    else:
        df = filter2_df
        
    try:
        temp = pd.DataFrame(df[f'filter{n}'][i])
        temp['kl_pk'] = df['kl_pk'][i]
        return_value = pd.merge(temp, df[i:i+1][['kl_pk', 'kl_name', 'similar_score']], how = 'left', on = 'kl_pk')
    except : 
        return_value = df[i:i+1][['kl_pk', 'kl_name', 'similar_score']]
        
    return return_value

In [ ]:
filter1_result = pd.DataFrame()
for i in range(0, len(filter1_df)):
    temp_df = dict_to_row(i, 1)
    filter1_result = pd.concat([filter1_result, temp_df], axis = 0)
    
filter2_result = pd.DataFrame()
for i in range(0, len(filter2_df)):
    temp_df = dict_to_row(i, 2)
    filter2_result = pd.concat([filter2_result, temp_df], axis = 0)    

정렬

In [ ]:
# 행 정렬
filter1_result.sort_values(['similar_score', 'score', 'kl_name', 'tags'], ascending = False, inplace = True)
filter2_result.sort_values(['similar_score', 'score', 'kl_name', 'tags'], ascending = False, inplace = True)

# 열 정렬
filter1_result = filter1_result[['kl_pk', 'kl_name', 'word', 'similar_score', 'score', 'tags']].reset_index(drop = True)
filter2_result = filter2_result[['kl_pk', 'kl_name', 'word', 'similar_score', 'score', 'tags']].reset_index(drop = True)

내보내기

In [ ]:
with pd.ExcelWriter('유사동의어 결과.xlsx') as writer:  
    keytalk_list.to_excel(writer, sheet_name='All')
    filter1_result.to_excel(writer, sheet_name='Filter1')
    filter2_result.to_excel(writer, sheet_name='Filter2') 